In [3]:
from qiskit import QuantumCircuit

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=3)

import time

In [4]:
# Create a quantum circuit with no parameters
qc = QuantumCircuit(2, 2)
qc.cx(0, 1)
qc.rz(0.1, 0)
qc.cx(0, 1)
qc.rz(0.2, 0)

print(qc)

          ┌─────────┐     ┌─────────┐
q_0: ──■──┤ Rz(0.1) ├──■──┤ Rz(0.2) ├
     ┌─┴─┐└─────────┘┌─┴─┐└─────────┘
q_1: ┤ X ├───────────┤ X ├───────────
     └───┘           └───┘           
c: 2/════════════════════════════════
                                     


In [5]:
t1 = time.time()
default_qc = pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

print(default_qc)

Time taken:  0.008243083953857422
OrderedDict({'rz': 1})
Number of 2-qubit gates:  0
Number of 1-qubit gates:  1
     ┌─────────┐
q_0: ┤ Rz(0.3) ├
     └─────────┘
q_1: ───────────
                
c: 2/═══════════
                


In [6]:
#Create a quantum circuit with parameters
from qiskit.circuit import Parameter

#Create parameter vector
p1 = Parameter("angle1")
p2 = Parameter("angle2")

pqc = QuantumCircuit(2, 2)
pqc.cx(0, 1)
pqc.rz(p1, 0)
pqc.cx(0, 1)
pqc.rz(p2, 0)

print(pqc)


          ┌────────────┐     ┌────────────┐
q_0: ──■──┤ Rz(angle1) ├──■──┤ Rz(angle2) ├
     ┌─┴─┐└────────────┘┌─┴─┐└────────────┘
q_1: ┤ X ├──────────────┤ X ├──────────────
     └───┘              └───┘              
c: 2/══════════════════════════════════════
                                           


In [7]:
t1 = time.time()
default_pqc = pm.run(pqc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_pqc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

print(default_pqc)

Time taken:  0.003178119659423828
OrderedDict({'cx': 2, 'rz': 2})
Number of 2-qubit gates:  2
Number of 1-qubit gates:  2
          ┌────────────┐     ┌────────────┐
q_0: ──■──┤ Rz(angle1) ├──■──┤ Rz(angle2) ├
     ┌─┴─┐└────────────┘┌─┴─┐└────────────┘
q_1: ┤ X ├──────────────┤ X ├──────────────
     └───┘              └───┘              
c: 2/══════════════════════════════════════
                                           


Testing UCC

In [8]:
import sys
sys.path.append('../')

from ucc.transpilers.ucc_defaults import UCCDefault1

ucc_transpiler = UCCDefault1()
transpiled_circuit = ucc_transpiler.run(pqc)

print(transpiled_circuit)

     ┌────────────┐┌────────────┐
q_0: ┤ Rz(angle1) ├┤ Rz(angle2) ├
     └────────────┘└────────────┘
q_1: ────────────────────────────
                                 
c: 2/════════════════════════════
                                 


In [9]:
from ucc import compile

ucc_qc, gate_counts = compile(pqc, mode="qiskit", get_gate_counts = True) 
gate_counts

OrderedDict([('cx', 2), ('rz', 2)])